In [63]:
#import libraries
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
import csv 
import lxml
from textblob import TextBlob
import regex as re

In [2]:
# get raw html
URL = "https://www.isaaa.org/gmapprovaldatabase/geneslist/default.asp"
gmo = requests.get(URL) 
soup = BeautifulSoup(gmo.content, "lxml")
# get text 
souptext = str(soup.get_text())

In [3]:
# get rid of leading and trailing whitespace
genetext_raw = souptext.splitlines() 

# remove leading and trailing whitespace
genetext = []
for element in genetext_raw: 
    genetext_entry = element.strip()
    genetext.append(genetext_entry)

# remove empty list entries 
while('' in genetext) : 
    genetext.remove('') 

# slice out only parts we want
genetext = genetext[34:-15]
print(genetext)

['Genes List', 'Gene', 'Product and Function', 'Trait: 2,4-D herbicide tolerance', 'aad-1', 'Source:', 'synthetic form of the aad-1 gene from Sphingobium herbicidovorans', 'Product:', 'aryloxyalkanoate dioxygenase 1 (AAD-1) protein', 'Function:', 'detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides', 'aad-12', 'Source:', 'Delftia acidovorans', 'Product:', 'aryloxyalkanoate di-oxygenase 12 (AAD-12) protein', 'Function:', 'catalyzes the side chain degradation of 2,4-D herbicide', 'Trait: Altered lignin production', 'ccomt (inverted repeat)', 'Source:', 'Medicago sativa (alfalfa)', 'Product:', 'dsRNA that suppresses endogenous S-adenosyl-L-methionine: trans-caffeoyl CoA 3-O-methyltransferase (CCOMT gene) RNA transcript levels via the RNA interference (RNAi) pathway', 'Function:', 'reduces content of guaiacyl (G) lignin', 'EgCAld5H', 'Source:', 'Eucalyptus grandis', 'Product:', 'CAld5H enzyme', 'Function:', 'regulates t

In [4]:
# iterate, +1 index for keyword? (trait inclusive, gene name is -1 source:)

In [5]:

traits = [info for info in genetext if "Trait:" in info]
traits

['Trait: 2,4-D herbicide tolerance',
 'Trait: Altered lignin production',
 'Trait: Anti-allergy',
 'Trait: Antibiotic resistance',
 'Trait: Coleopteran insect resistance',
 'Trait: Delayed fruit softening',
 'Trait: Delayed ripening/senescence',
 'Trait: Dicamba herbicide tolerance',
 'Trait: Drought stress tolerance',
 'Trait: Enhanced Photosynthesis/Yield',
 'Trait: Enhanced Provitamin A Content',
 'Trait: Fertility restoration',
 'Trait: Foliar Late Blight Resistance',
 'Trait: Glufosinate herbicide tolerance',
 'Trait: Glyphosate herbicide tolerance',
 'Trait: Hemipteran Insect Resistance',
 'Trait: Imazamox herbicide tolerance',
 'Trait: Increased Ear Biomass',
 'Trait: Inhibited stem elongation (dwarfed)',
 'Trait: Isoxaflutole herbicide tolerance',
 'Trait: Late blight disease resistance',
 'Trait: Lepidopteran insect resistance',
 'Trait: Low Gossypol',
 'Trait: Lowered Free Asparagine',
 'Trait: Lowered Reducing Sugars',
 'Trait: Male sterility',
 'Trait: Mannose metabolism',


In [ ]:

genenames = []
for i in range(1, len(genetext)): 
    if genetext[i] == 'Source:': 
        genenames.append(genetext[i-1])
genenames

In [46]:
sublist = []
traitinfo = []
for info in genetext: 
    if info in traits: 
        if sublist:
            traitinfo.append(sublist)
        sublist = [info]
    else: 
        sublist.append(info)
traitinfo.append(sublist)


['Trait: Low Gossypol',
 'dCS',
 'Source:',
 'Gossypium hirsutum L.',
 'Product:',
 'dsRNA that suppresses the expression of d-cadinene synthase gene that encode d-cadinene synthase, a key enzyme involved in gossypolbiosynthesis, thru RNAi pathway',
 'Function:',
 'silence the endogenous dCS genes']

In [ ]:
### DIRTY FIX FOR dCS
traitinfo[23][5:6] = [''.join(traitinfo[23][5:7])]
del traitinfo[23][6]
traitinfo[23]

In [47]:
data = [] 

for sublist in traitinfo: 
    for item in sublist:
        if "Trait:" in item:
            trait = item
        if item == "Source:": 
            gene = sublist[sublist.index(item)-1]
        if item == "Source:": 
            source = sublist[sublist.index(item)+1]
        if item == 'Product:': 
            product = sublist[sublist.index(item)+1]
        if item == 'Function:':
            function = sublist[sublist.index(item)+1]

            row = {
                'Gene': gene,
                'Source': source,
                'Product': product, 
                'Function': function
            }
            
            if row:
            
                info = trait, row

            data.append(info)
data
    

[('Trait: 2,4-D herbicide tolerance',
  {'Gene': 'aad-1',
   'Source': 'synthetic form of the aad-1 gene from Sphingobium herbicidovorans',
   'Product': 'aryloxyalkanoate dioxygenase 1 (AAD-1) protein',
   'Function': 'detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides'}),
 ('Trait: 2,4-D herbicide tolerance',
  {'Gene': 'aad-1',
   'Source': 'synthetic form of the aad-1 gene from Sphingobium herbicidovorans',
   'Product': 'aryloxyalkanoate dioxygenase 1 (AAD-1) protein',
   'Function': 'detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides'}),
 ('Trait: Altered lignin production',
  {'Gene': 'ccomt (inverted repeat)',
   'Source': 'Medicago sativa (alfalfa)',
   'Product': 'dsRNA that suppresses endogenous S-adenosyl-L-methionine: trans-caffeoyl CoA 3-O-methyltransferase (CCOMT gene) RNA transcript levels via the RNA interference (RNAi) pathway

In [48]:
gene = []
geneinfo = []
for sublist in traitinfo: 
    for info in sublist:
        if info in genenames: 
            if gene:
                geneinfo.append(gene)
            gene = [info]
        else: 
            gene.append(info)
    geneinfo.append(gene)
    print(geneinfo)

[['Genes List', 'Gene', 'Product and Function']]
[['Genes List', 'Gene', 'Product and Function', 'Trait: 2,4-D herbicide tolerance'], ['Genes List', 'Gene', 'Product and Function', 'Trait: 2,4-D herbicide tolerance'], ['aad-1', 'Source:', 'synthetic form of the aad-1 gene from Sphingobium herbicidovorans', 'Product:', 'aryloxyalkanoate dioxygenase 1 (AAD-1) protein', 'Function:', 'detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides'], ['aad-12', 'Source:', 'Delftia acidovorans', 'Product:', 'aryloxyalkanoate di-oxygenase 12 (AAD-12) protein', 'Function:', 'catalyzes the side chain degradation of 2,4-D herbicide']]
[['Genes List', 'Gene', 'Product and Function', 'Trait: 2,4-D herbicide tolerance'], ['Genes List', 'Gene', 'Product and Function', 'Trait: 2,4-D herbicide tolerance'], ['aad-1', 'Source:', 'synthetic form of the aad-1 gene from Sphingobium herbicidovorans', 'Product:', 'aryloxyalkanoate dioxygenase 1 (AAD

In [49]:
traitinfo

[['Genes List', 'Gene', 'Product and Function'],
 ['Trait: 2,4-D herbicide tolerance',
  'aad-1',
  'Source:',
  'synthetic form of the aad-1 gene from Sphingobium herbicidovorans',
  'Product:',
  'aryloxyalkanoate dioxygenase 1 (AAD-1) protein',
  'Function:',
  'detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides',
  'aad-12',
  'Source:',
  'Delftia acidovorans',
  'Product:',
  'aryloxyalkanoate di-oxygenase 12 (AAD-12) protein',
  'Function:',
  'catalyzes the side chain degradation of 2,4-D herbicide'],
 ['Trait: Altered lignin production',
  'ccomt (inverted repeat)',
  'Source:',
  'Medicago sativa (alfalfa)',
  'Product:',
  'dsRNA that suppresses endogenous S-adenosyl-L-methionine: trans-caffeoyl CoA 3-O-methyltransferase (CCOMT gene) RNA transcript levels via the RNA interference (RNAi) pathway',
  'Function:',
  'reduces content of guaiacyl (G) lignin',
  'EgCAld5H',
  'Source:',
  'Eucalyptus grandis',

In [50]:
df = pd.DataFrame(traitinfo)
df = pd.DataFrame.dropna(df, how = 'all')
df = df.drop(0,0)
df

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
1,"Trait: 2,4-D herbicide tolerance",aad-1,Source:,synthetic form of the aad-1 gene from Sphingob...,Product:,aryloxyalkanoate dioxygenase 1 (AAD-1) protein,Function:,"detoxifies 2,4-D herbicide by side-chain degra...",aad-12,Source:,...,None,None,None,None,None,None,None,None,None,None
2,Trait: Altered lignin production,ccomt (inverted repeat),Source:,Medicago sativa (alfalfa),Product:,dsRNA that suppresses endogenous S-adenosyl-L-...,Function:,reduces content of guaiacyl (G) lignin,EgCAld5H,Source:,...,None,None,None,None,None,None,None,None,None,None
3,Trait: Anti-allergy,7crp,Source:,synthetic form of tolerogenic protein from Cry...,Product:,modified cry j 1 and cry j 2 pollen antigens c...,Function:,triggers mucosal immune tolerance to cedar pol...,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Trait: Antibiotic resistance,aad,Source:,Escherichia coli,Product:,3''(9)-O-aminoglycoside adenylyltransferase en...,Function:,allows selection for resistance to aminoglycos...,aph4 (hpt),Source:,...,None,None,None,None,None,None,None,None,None,None
5,Trait: Coleopteran insect resistance,cry34Ab1,Source:,Bacillus thuringiensis strain PS149B1,Product:,Cry34Ab1 delta-endotoxin,Function:,confers resistance to coleopteran insects part...,cry35Ab1,Source:,...,None,None,None,None,None,None,None,None,None,None
6,Trait: Delayed fruit softening,pg (sense or antisense),Source:,Lycopersicon esculentum,Product:,no functional polygalacturonase enzyme is prod...,Function:,inhibits the production of polygalacturonase e...,None,None,...,None,None,None,None,None,None,None,None,None,None
7,Trait: Delayed ripening/senescence,acc,Source:,Ananas comosus,Product:,1-aminocyclopropane-1-carboxylic acid synthase,Function:,involved in catalyzing the penultimate step in...,acc (truncated),Source:,...,None,None,None,None,None,None,None,None,None,None
8,Trait: Dicamba herbicide tolerance,dmo,Source:,Stenotrophomonas maltophilia strain DI-6,Product:,dicamba mono-oxygenase enzyme,Function:,confers tolerance to the herbicide dicamba (2-...,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Trait: Drought stress tolerance,cspB,Source:,Bacillus subtilis,Product:,cold shock protein B,Function:,maintains normal cellular functions under wate...,EcBetA,Source:,...,None,None,None,None,None,None,None,None,None,None
10,Trait: Enhanced Photosynthesis/Yield,bbx32,Source:,Arabidopsis thaliana,Product:,protein that interacts with one or more endoge...,Function:,to modulate plant's diurnal biology and to enh...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [51]:
data_full = [] ### main list in which we will populate rows
for index, row in df.iterrows(): ### you can use df.iterrows() here
    count = 1  ### Initiating the count
    ### Now we want to run a while loop for each row until it encounters None
    while (row[count] is not None) & (count < df.shape[1]):
        
        data_part = [] ### we will be adding this list to all_rows list defined earlier
        
        keywords = ["Source:", "Function:", "Product:"] #error handling for empty function:
        
        if row[count] in keywords: #DIRTY FIX
            count = count - 1 
            
        data_part.append(row[0]) #trait 
        data_part.append(row[count]) # gene
        data_part.append(row[count + 2]) # source
        data_part.append(row[count + 4]) # product
        data_part.append(row[count + 6]) # function

        data_full.append(data_part) # append to main list
        
        count = count + 7 ### Because each gene occupies 7 columns
        
        # break statement 
        if count >= df.shape[1]:
            break
        
### Finally, we wanna use main list to convert to dataframe
traitsdata = pd.DataFrame(data_full) 

In [52]:
traitsdata

,0,1,2,3,4
0,"Trait: 2,4-D herbicide tolerance",aad-1,synthetic form of the aad-1 gene from Sphingob...,aryloxyalkanoate dioxygenase 1 (AAD-1) protein,"detoxifies 2,4-D herbicide by side-chain degra..."
1,"Trait: 2,4-D herbicide tolerance",aad-12,Delftia acidovorans,aryloxyalkanoate di-oxygenase 12 (AAD-12) protein,"catalyzes the side chain degradation of 2,4-D ..."
2,Trait: Altered lignin production,ccomt (inverted repeat),Medicago sativa (alfalfa),dsRNA that suppresses endogenous S-adenosyl-L-...,reduces content of guaiacyl (G) lignin
3,Trait: Altered lignin production,EgCAld5H,Eucalyptus grandis,CAld5H enzyme,regulates the syringyl monolignol pathway;
4,Trait: Anti-allergy,7crp,synthetic form of tolerogenic protein from Cry...,modified cry j 1 and cry j 2 pollen antigens c...,triggers mucosal immune tolerance to cedar pol...
...,...,...,...,...,...
141,Trait: Viral disease resistance,wmv_cp,Watermelon Mosaic Potyvirus 2 (WMV2),coat protein of watermelon mosaic potyvirus 2 ...,confers resistance to watermelon mosaic potyvi...
142,Trait: Viral disease resistance,zymv_cp,Zucchini Yellow Mosaic Potyvirus (ZYMV),coat protein of zucchini yellow mosaic potyvir...,confers resistance to zucchini yellow mosaic p...
143,Trait: Visual marker,dsRed2,Discosoma sp.,red fluorescent protein,"produces red stain on transformed tissue, whic..."
144,Trait: Visual marker,uidA,Escherichia coli,beta-D-glucuronidase (GUS) enzyme,produces blue stain on treated transformed tis...


In [60]:
traitsdata[0] = traitsdata[0].map(lambda x: x.lstrip('Trait: '))
traitsdata.columns = ['Trait', "Gene","Product", "Source", "Function"]

In [61]:
traitsdata

,Trait,Gene,Product,Source,Function
0,"2,4-D herbicide tolerance",aad-1,synthetic form of the aad-1 gene from Sphingob...,aryloxyalkanoate dioxygenase 1 (AAD-1) protein,"detoxifies 2,4-D herbicide by side-chain degra..."
1,"2,4-D herbicide tolerance",aad-12,Delftia acidovorans,aryloxyalkanoate di-oxygenase 12 (AAD-12) protein,"catalyzes the side chain degradation of 2,4-D ..."
2,Altered lignin production,ccomt (inverted repeat),Medicago sativa (alfalfa),dsRNA that suppresses endogenous S-adenosyl-L-...,reduces content of guaiacyl (G) lignin
3,Altered lignin production,EgCAld5H,Eucalyptus grandis,CAld5H enzyme,regulates the syringyl monolignol pathway;
4,Anti-allergy,7crp,synthetic form of tolerogenic protein from Cry...,modified cry j 1 and cry j 2 pollen antigens c...,triggers mucosal immune tolerance to cedar pol...
...,...,...,...,...,...
141,Viral disease resistance,wmv_cp,Watermelon Mosaic Potyvirus 2 (WMV2),coat protein of watermelon mosaic potyvirus 2 ...,confers resistance to watermelon mosaic potyvi...
142,Viral disease resistance,zymv_cp,Zucchini Yellow Mosaic Potyvirus (ZYMV),coat protein of zucchini yellow mosaic potyvir...,confers resistance to zucchini yellow mosaic p...
143,Visual marker,dsRed2,Discosoma sp.,red fluorescent protein,"produces red stain on transformed tissue, whic..."
144,Visual marker,uidA,Escherichia coli,beta-D-glucuronidase (GUS) enzyme,produces blue stain on treated transformed tis...


In [ ]:
# using blobtext
souptext = str(soup.get_text())
blobtext = TextBlob(souptext)
blobtext2 = str(blobtext)
print(blobtext2, file=open("blobtext.txt", "a")) # write to file

In [ ]:
blobtext2

In [ ]:
# dictionary for regex matching
gene_dict = {
    'trait': re.compile(r'Trait: (?P<trait>.*)\n'),
    'gene' : re.compile(r'\n(?P<gene>.*\s)\n Source:'),
    'source': re.compile(r'Source:\n(?P<source>.*)\n'), 
    'product': re.compile(r'Product:\n(?P<product>.*)\n'),
    'function': re.compile(r'Function:\n(?P<function>.*)\n')
}

In [ ]:
# match lines to regex
def parseinfo(text): 
    for info, regex in gene_dict.items():
        match = regex.search(text)
        if match: 
            return info, match
    return None, None

In [ ]:
# match lines to regex
def parseinfo(text): 
    for info, regex in gene_dict.items():
        match = re.search(regex, text)
        if match: 
            return info, match
        else: 
            return None, None

In [ ]:
parseinfo(blobtext2) #test

In [ ]:
with open('./blobtext.txt', 'r') as file: 
    dat = []
    line = file.readline()
    for line in file:
        info, match = parseinfo(line)
        if info == 'trait': 
            trait = re.match('trait', line)
            if trait not None:
            dat.append(trait)

In [ ]:
# parse blobtext file 


######### needs debugging!! 
def parsefile(file): 
    data = []
    with open(file, 'r') as file: 
        
        line = file.readline()
        
        while line: 
            
            info, match = parseinfo(line)
            
            if info == 'trait': 
                trait = match.group('trait')
                
            if info == 'gene': 
                gene = match.group('gene')
                
            if info == 'source': 
                source = match.group('source')
                
            if info == 'product':
                product = match.group('product')
                
            if info == 'function': 
                function = match.group('function')

                while line.strip():
                    row = {
                        'Trait': trait, 
                        'Gene': gene,
                        'Source': source,
                        'Product': product,
                        'Function': function
                    }

                    data.append(row)

        data = pd.DataFrame(data)
        data.set_index(['Trait', 'Gene'], inplace = True)
        
        data = data.groupby(level = data.index.names).first()
    file.close()    
    return data

In [ ]:
parsefile('./blobtext.txt')

## Scrapwork

In [ ]:
#using pure webscraping

In [ ]:
str_gt = str(genetext_raw)
df = pd.DataFrame([x.split(',') for x in str_gt.split('\n')])
pd.DataFrame.dropna(df, how = 'all')

In [ ]:
gene_names = []
gmo_tds = soup("tr", class_="smalltext")
for td in gmo_tds:
    gene_names.append(td.find("a").contents[0])
gene_names

In [ ]:
soup = BeautifulSoup('<td bgcolor="#F7F7F7" valign="top"><strong>Product:</strong> CAld5H enzyme <br/>')
tag = soup.b
type(tag)

In [ ]:
classes = []
for element in soup.find_all(class_=True):
    classes.extend(element["class"])
print(classes)

In [100]:
approvaldata = pd.read_csv("./foodapprovals.csv")

In [101]:
approvaldata = approvaldata.rename(columns=approvaldata.iloc[0])
approvaldata = approvaldata.drop(0,0)

In [102]:
approvaldata

,Rank,Country,1992–2003,2004–2014,Total
1,1,United States,96,60,156
2,2,Canada,56,67,123
3,3,Japan,5,115,120
4,4,Brazil*,1,38,39
5,5,Argentina*,7,29,36
6,6,Australia,21,11,32
7,7,South Africa*,10,9,19
8,8,Uruguay*,2,16,18
9,9,Colombia*,1,13,14
10,10,Costa Rica*,1,14,15


In [104]:
approvaldata.to_csv("./foodapprovals.csv", index = False)